In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model


print("TensorFlow Version:", tf.__version__)


# LOAD LABELED IMAGES


def load_labeled_images(root_folder, image_size=(224, 224)):
   """Loads labeled dataset images without using labels (for soft label generation)."""
   images_list = []
  
   for label in ["1", "2", "3"]:
       label_dir = os.path.join(root_folder, label)
       if not os.path.isdir(label_dir):
           continue


       for fn in os.listdir(label_dir):
           if fn.lower().endswith((".jpg", ".jpeg", ".png")):
               img_path = os.path.join(label_dir, fn)
               try:
                   image = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
                   image = tf.keras.preprocessing.image.img_to_array(image) / 255.0  # Normalize
                   images_list.append(image)
               except Exception as e:
                   print(f"⚠️ [Skip] {img_path}: {e}")


   if len(images_list) == 0:
       raise ValueError("No images were loaded! Check dataset structure.")


   return np.stack(images_list, axis=0)


# LOAD PRETRAINED CLASSIFIER MODEL


def load_pretrained_teacher_classifier(model_path):
   """Loads the already trained teacher classifier (which includes the classification head)."""
   print(" Loading trained teacher classifier model...")
   return tf.keras.models.load_model(model_path)  # No need to modify the model structure


# GENERATE SOFT LABELS WITH TEMPERATURE SCALING


def generate_soft_labels(teacher_classifier_path, labeled_data_path, output_file="soft_labels.npz", batch_size=32, temperature=3.0):
   """Generates softened soft labels using temperature scaling."""
  
   teacher_model = load_pretrained_teacher_classifier(teacher_classifier_path)


   print("Loading labeled dataset for soft label generation...")
   images = load_labeled_images(labeled_data_path)  # Load images


   dataset = tf.data.Dataset.from_tensor_slices(images)
   dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


   # Generate raw soft labels (logits)
   print("Generating raw soft labels...")
   raw_logits = teacher_model.predict(dataset, verbose=1)  # Before softmax


   #possibly look into applying this tempreature scaling in the classification head code , as it may be unecessary to add another softmax layer


   # Apply Temperature Scaling
   print(f"Applying Temperature Scaling (T={temperature})...")
   soft_labels = tf.nn.softmax(raw_logits / temperature).numpy()


   # Save soft labels
   print(f"Saving soft labels to {output_file}...")
   np.savez(output_file, soft_labels=soft_labels)


   print(" Soft label generation complete! Labels saved as 'soft_labels.npz'.")


   return soft_labels




# RUN SOFT LABEL GENERATION
if __name__ == "__main__":
   labeled_data_path = "/Users/morgan/Desktop/BT-ClassificationDissCode/LabelledFigshare"
   teacher_classifier_path = "teacher_classifier_modelv5.keras"  # Directly use trained classifier


   generate_soft_labels(teacher_classifier_path, labeled_data_path)